In [1]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import cv2
from sklearn.model_selection import train_test_split
import os
import pickle
from PIL import Image
from resizeimage import resizeimage

In [2]:
device = torch.device('cpu')  # sad reax only

In [3]:
x = []  # image data, start as list
y = []  # output class data, start as list

for folder in os.listdir("./pictures/"):
    for file in os.listdir("./pictures/" + folder + "/"):
        im = cv2.imread("./pictures/" + folder + "/" + file)
        im = np.reshape(im, (3, 256, 144))
        x.append(im)
        out = np.zeros(9)
        out[int(folder)] = 1
        # out = int(folder)
        y.append(out)

x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.002, random_state=0)

In [4]:
class TrainDataset(Dataset):
    def __init__(self):
        self.len = x_train.shape[0]
        self.x_data = torch.from_numpy(x_train).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_train).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [5]:
class TestDataset(Dataset):
    def __init__(self):
        self.len = x_test.shape[0]
        self.x_data = torch.from_numpy(x_test).type(torch.FloatTensor)
        self.y_data = torch.from_numpy(y_test).type(torch.LongTensor)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [6]:
class Net(nn.Module):
    def __init__(self, num_classes=9):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, 5, 1)
        self.conv2 = nn.Conv2d(5, 10, 5, 1)
        self.fc1 = nn.Linear(20130, 10)
        self.fc2 = nn.Linear(10, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 20130)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [7]:
learning_rate = 0.002
model = Net()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [8]:
def train(epoch, train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        #print(data)
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, torch.max(target, 1)[1])
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.data))
    if epoch == 10:
        pkl_filename = "pickle_model.pkl"
        with open(pkl_filename, 'wb') as file:
            pickle.dump(model, file)

In [ ]:
if __name__ == '__main__':
    train_dataset = TrainDataset()
    train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

    test_dataset = TestDataset()
    test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)

    """for epoch in range(5):
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            inputs, labels = Variable(inputs), Variable(labels)
            print(epoch, i, "inputs", inputs.data, "labels", labels.data)"""

    for epoch in range(1, 11):
        train(epoch, train_loader)

        # test(test_loader)

Train Epoch: 1 [0/4115 (0%)]	Loss: 7.883359
Train Epoch: 1 [10/4115 (0%)]	Loss: 17.925396
Train Epoch: 1 [20/4115 (0%)]	Loss: 2.317638
Train Epoch: 1 [30/4115 (1%)]	Loss: 2.308491
Train Epoch: 1 [40/4115 (1%)]	Loss: 2.184898
Train Epoch: 1 [50/4115 (1%)]	Loss: 2.296830
Train Epoch: 1 [60/4115 (1%)]	Loss: 2.258298
Train Epoch: 1 [70/4115 (2%)]	Loss: 2.307984
Train Epoch: 1 [80/4115 (2%)]	Loss: 2.249573
Train Epoch: 1 [90/4115 (2%)]	Loss: 2.181196
Train Epoch: 1 [100/4115 (2%)]	Loss: 2.178076
Train Epoch: 1 [110/4115 (3%)]	Loss: 2.304269
Train Epoch: 1 [120/4115 (3%)]	Loss: 2.283927
Train Epoch: 1 [130/4115 (3%)]	Loss: 2.280041
Train Epoch: 1 [140/4115 (3%)]	Loss: 2.217771
Train Epoch: 1 [150/4115 (4%)]	Loss: 2.162474
Train Epoch: 1 [160/4115 (4%)]	Loss: 2.270305
Train Epoch: 1 [170/4115 (4%)]	Loss: 2.299798
Train Epoch: 1 [180/4115 (4%)]	Loss: 2.211418
Train Epoch: 1 [190/4115 (5%)]	Loss: 2.201531
Train Epoch: 1 [200/4115 (5%)]	Loss: 2.122538
Train Epoch: 1 [210/4115 (5%)]	Loss: 2.26358

Train Epoch: 1 [1730/4115 (42%)]	Loss: 2.074649
Train Epoch: 1 [1740/4115 (42%)]	Loss: 2.122252
Train Epoch: 1 [1750/4115 (43%)]	Loss: 2.116087
Train Epoch: 1 [1760/4115 (43%)]	Loss: 2.108656
Train Epoch: 1 [1770/4115 (43%)]	Loss: 2.175819
Train Epoch: 1 [1780/4115 (43%)]	Loss: 2.344842
Train Epoch: 1 [1790/4115 (43%)]	Loss: 2.242212
Train Epoch: 1 [1800/4115 (44%)]	Loss: 2.354177
Train Epoch: 1 [1810/4115 (44%)]	Loss: 2.156775
Train Epoch: 1 [1820/4115 (44%)]	Loss: 2.105045
Train Epoch: 1 [1830/4115 (44%)]	Loss: 2.175770
Train Epoch: 1 [1840/4115 (45%)]	Loss: 2.071345
Train Epoch: 1 [1850/4115 (45%)]	Loss: 2.124500
Train Epoch: 1 [1860/4115 (45%)]	Loss: 2.159873
Train Epoch: 1 [1870/4115 (45%)]	Loss: 2.115118
Train Epoch: 1 [1880/4115 (46%)]	Loss: 2.113207
Train Epoch: 1 [1890/4115 (46%)]	Loss: 2.171692
Train Epoch: 1 [1900/4115 (46%)]	Loss: 2.141174
Train Epoch: 1 [1910/4115 (46%)]	Loss: 2.086445
Train Epoch: 1 [1920/4115 (47%)]	Loss: 2.136152
Train Epoch: 1 [1930/4115 (47%)]	Loss: 2

Train Epoch: 1 [3430/4115 (83%)]	Loss: 2.048805
Train Epoch: 1 [3440/4115 (84%)]	Loss: 2.120656
Train Epoch: 1 [3450/4115 (84%)]	Loss: 2.070407
Train Epoch: 1 [3460/4115 (84%)]	Loss: 2.294081
Train Epoch: 1 [3470/4115 (84%)]	Loss: 2.115103
Train Epoch: 1 [3480/4115 (85%)]	Loss: 2.074508
Train Epoch: 1 [3490/4115 (85%)]	Loss: 2.027386
Train Epoch: 1 [3500/4115 (85%)]	Loss: 2.044218
Train Epoch: 1 [3510/4115 (85%)]	Loss: 2.108320
Train Epoch: 1 [3520/4115 (86%)]	Loss: 2.523929
Train Epoch: 1 [3530/4115 (86%)]	Loss: 2.042889
Train Epoch: 1 [3540/4115 (86%)]	Loss: 2.078963
Train Epoch: 1 [3550/4115 (86%)]	Loss: 2.073256
Train Epoch: 1 [3560/4115 (87%)]	Loss: 2.026574
Train Epoch: 1 [3570/4115 (87%)]	Loss: 2.095600
Train Epoch: 1 [3580/4115 (87%)]	Loss: 2.011995
Train Epoch: 1 [3590/4115 (87%)]	Loss: 2.097361
Train Epoch: 1 [3600/4115 (87%)]	Loss: 2.262512
Train Epoch: 1 [3610/4115 (88%)]	Loss: 2.545338
Train Epoch: 1 [3620/4115 (88%)]	Loss: 2.079416
Train Epoch: 1 [3630/4115 (88%)]	Loss: 2

Train Epoch: 2 [1040/4115 (25%)]	Loss: 2.567742
Train Epoch: 2 [1050/4115 (26%)]	Loss: 2.395588
Train Epoch: 2 [1060/4115 (26%)]	Loss: 2.051146
Train Epoch: 2 [1070/4115 (26%)]	Loss: 2.553455
Train Epoch: 2 [1080/4115 (26%)]	Loss: 2.545382
Train Epoch: 2 [1090/4115 (26%)]	Loss: 2.050928
Train Epoch: 2 [1100/4115 (27%)]	Loss: 2.228778
Train Epoch: 2 [1110/4115 (27%)]	Loss: 2.011823
Train Epoch: 2 [1120/4115 (27%)]	Loss: 2.226262
Train Epoch: 2 [1130/4115 (27%)]	Loss: 2.039230
Train Epoch: 2 [1140/4115 (28%)]	Loss: 2.523184
Train Epoch: 2 [1150/4115 (28%)]	Loss: 2.050556
Train Epoch: 2 [1160/4115 (28%)]	Loss: 2.523385
Train Epoch: 2 [1170/4115 (28%)]	Loss: 2.023756
Train Epoch: 2 [1180/4115 (29%)]	Loss: 2.351344
Train Epoch: 2 [1190/4115 (29%)]	Loss: 2.212758
Train Epoch: 2 [1200/4115 (29%)]	Loss: 2.033890
Train Epoch: 2 [1210/4115 (29%)]	Loss: 2.029901
Train Epoch: 2 [1220/4115 (30%)]	Loss: 2.030827
Train Epoch: 2 [1230/4115 (30%)]	Loss: 2.028452
Train Epoch: 2 [1240/4115 (30%)]	Loss: 2

Train Epoch: 2 [2740/4115 (67%)]	Loss: 2.266079
Train Epoch: 2 [2750/4115 (67%)]	Loss: 2.109013
Train Epoch: 2 [2760/4115 (67%)]	Loss: 2.082600
Train Epoch: 2 [2770/4115 (67%)]	Loss: 1.976402
Train Epoch: 2 [2780/4115 (68%)]	Loss: 2.274515
Train Epoch: 2 [2790/4115 (68%)]	Loss: 2.477916
Train Epoch: 2 [2800/4115 (68%)]	Loss: 1.971819
Train Epoch: 2 [2810/4115 (68%)]	Loss: 2.072602
Train Epoch: 2 [2820/4115 (69%)]	Loss: 2.126667
Train Epoch: 2 [2830/4115 (69%)]	Loss: 2.113538
Train Epoch: 2 [2840/4115 (69%)]	Loss: 2.277607
Train Epoch: 2 [2850/4115 (69%)]	Loss: 2.271952
Train Epoch: 2 [2860/4115 (70%)]	Loss: 2.079554
Train Epoch: 2 [2870/4115 (70%)]	Loss: 2.076798
Train Epoch: 2 [2880/4115 (70%)]	Loss: 2.084012
Train Epoch: 2 [2890/4115 (70%)]	Loss: 1.992492
Train Epoch: 2 [2900/4115 (70%)]	Loss: 2.102399
Train Epoch: 2 [2910/4115 (71%)]	Loss: 2.068208
Train Epoch: 2 [2920/4115 (71%)]	Loss: 2.065822
Train Epoch: 2 [2930/4115 (71%)]	Loss: 2.267213
Train Epoch: 2 [2940/4115 (71%)]	Loss: 2

Train Epoch: 3 [340/4115 (8%)]	Loss: 2.090203
Train Epoch: 3 [350/4115 (9%)]	Loss: 2.039075
Train Epoch: 3 [360/4115 (9%)]	Loss: 2.085175
Train Epoch: 3 [370/4115 (9%)]	Loss: 2.034018
Train Epoch: 3 [380/4115 (9%)]	Loss: 2.034806
Train Epoch: 3 [390/4115 (9%)]	Loss: 2.374552
Train Epoch: 3 [400/4115 (10%)]	Loss: 2.022457
Train Epoch: 3 [410/4115 (10%)]	Loss: 2.013675
Train Epoch: 3 [420/4115 (10%)]	Loss: 2.027698
Train Epoch: 3 [430/4115 (10%)]	Loss: 1.997792
Train Epoch: 3 [440/4115 (11%)]	Loss: 1.999051
Train Epoch: 3 [450/4115 (11%)]	Loss: 2.037947
Train Epoch: 3 [460/4115 (11%)]	Loss: 2.033518
Train Epoch: 3 [470/4115 (11%)]	Loss: 1.981147
Train Epoch: 3 [480/4115 (12%)]	Loss: 2.040146
Train Epoch: 3 [490/4115 (12%)]	Loss: 2.362259
Train Epoch: 3 [500/4115 (12%)]	Loss: 1.971465
Train Epoch: 3 [510/4115 (12%)]	Loss: 2.038765
Train Epoch: 3 [520/4115 (13%)]	Loss: 2.033487
Train Epoch: 3 [530/4115 (13%)]	Loss: 2.111701
Train Epoch: 3 [540/4115 (13%)]	Loss: 2.019092
Train Epoch: 3 [550

Train Epoch: 3 [2060/4115 (50%)]	Loss: 2.332069
Train Epoch: 3 [2070/4115 (50%)]	Loss: 2.516176
Train Epoch: 3 [2080/4115 (51%)]	Loss: 2.327948
Train Epoch: 3 [2090/4115 (51%)]	Loss: 2.570770
Train Epoch: 3 [2100/4115 (51%)]	Loss: 2.031837
Train Epoch: 3 [2110/4115 (51%)]	Loss: 2.017202
Train Epoch: 3 [2120/4115 (52%)]	Loss: 2.015111
Train Epoch: 3 [2130/4115 (52%)]	Loss: 2.020606
Train Epoch: 3 [2140/4115 (52%)]	Loss: 2.317202
Train Epoch: 3 [2150/4115 (52%)]	Loss: 2.107684
Train Epoch: 3 [2160/4115 (52%)]	Loss: 2.012015
Train Epoch: 3 [2170/4115 (53%)]	Loss: 2.020217
Train Epoch: 3 [2180/4115 (53%)]	Loss: 2.577430
Train Epoch: 3 [2190/4115 (53%)]	Loss: 2.071494
Train Epoch: 3 [2200/4115 (53%)]	Loss: 2.071751
Train Epoch: 3 [2210/4115 (54%)]	Loss: 2.316805
Train Epoch: 3 [2220/4115 (54%)]	Loss: 2.106436
Train Epoch: 3 [2230/4115 (54%)]	Loss: 2.021957
Train Epoch: 3 [2240/4115 (54%)]	Loss: 2.010978
Train Epoch: 3 [2250/4115 (55%)]	Loss: 2.314250
Train Epoch: 3 [2260/4115 (55%)]	Loss: 2

Train Epoch: 3 [3760/4115 (91%)]	Loss: 2.034305
Train Epoch: 3 [3770/4115 (92%)]	Loss: 2.099686
Train Epoch: 3 [3780/4115 (92%)]	Loss: 2.098390
Train Epoch: 3 [3790/4115 (92%)]	Loss: 2.097360
Train Epoch: 3 [3800/4115 (92%)]	Loss: 2.094571
Train Epoch: 3 [3810/4115 (93%)]	Loss: 2.275237
Train Epoch: 3 [3820/4115 (93%)]	Loss: 2.355531
Train Epoch: 3 [3830/4115 (93%)]	Loss: 2.269917
Train Epoch: 3 [3840/4115 (93%)]	Loss: 2.518799
Train Epoch: 3 [3850/4115 (94%)]	Loss: 2.100803
Train Epoch: 3 [3860/4115 (94%)]	Loss: 2.353904
Train Epoch: 3 [3870/4115 (94%)]	Loss: 1.980663
Train Epoch: 3 [3880/4115 (94%)]	Loss: 2.044451
Train Epoch: 3 [3890/4115 (95%)]	Loss: 2.047951
Train Epoch: 3 [3900/4115 (95%)]	Loss: 1.969691
Train Epoch: 3 [3910/4115 (95%)]	Loss: 2.503643
Train Epoch: 3 [3920/4115 (95%)]	Loss: 2.497242
Train Epoch: 3 [3930/4115 (96%)]	Loss: 2.029883
Train Epoch: 3 [3940/4115 (96%)]	Loss: 2.491009
Train Epoch: 3 [3950/4115 (96%)]	Loss: 2.061749
Train Epoch: 3 [3960/4115 (96%)]	Loss: 2

Train Epoch: 4 [1370/4115 (33%)]	Loss: 2.074163
Train Epoch: 4 [1380/4115 (34%)]	Loss: 2.085013
Train Epoch: 4 [1390/4115 (34%)]	Loss: 2.364031
Train Epoch: 4 [1400/4115 (34%)]	Loss: 2.071683
Train Epoch: 4 [1410/4115 (34%)]	Loss: 2.463602
Train Epoch: 4 [1420/4115 (35%)]	Loss: 2.063709
Train Epoch: 4 [1430/4115 (35%)]	Loss: 2.080121
Train Epoch: 4 [1440/4115 (35%)]	Loss: 2.206027
Train Epoch: 4 [1450/4115 (35%)]	Loss: 2.084785
Train Epoch: 4 [1460/4115 (35%)]	Loss: 2.095598
Train Epoch: 4 [1470/4115 (36%)]	Loss: 2.359185
Train Epoch: 4 [1480/4115 (36%)]	Loss: 2.046501
Train Epoch: 4 [1490/4115 (36%)]	Loss: 2.041615
Train Epoch: 4 [1500/4115 (36%)]	Loss: 2.470093
Train Epoch: 4 [1510/4115 (37%)]	Loss: 2.030119
Train Epoch: 4 [1520/4115 (37%)]	Loss: 2.540089
Train Epoch: 4 [1530/4115 (37%)]	Loss: 2.103445
Train Epoch: 4 [1540/4115 (37%)]	Loss: 2.080728
Train Epoch: 4 [1550/4115 (38%)]	Loss: 2.225960
Train Epoch: 4 [1560/4115 (38%)]	Loss: 2.478782
Train Epoch: 4 [1570/4115 (38%)]	Loss: 2

In [ ]:
feedback = ["Move down and right", "Move down", "Move down and left", "Move right", "Perfect!", "Move left", "Move up and right", "Move up", "Move right"]
while True:
    for file in os.listdir("../../../../Downloads/"):
        l = []
        if 'ss_file' in file:
            with open("../../../../Downloads/" + file, 'r+b') as f:
                with Image.open(f) as image:
                    resized_image = resizeimage.resize_cover(image, [256, 144])
                    resized_image.save("../../../../Downloads/" + file, image.format)
            im = cv2.imread("../../../../Downloads/" + file)
            im = np.reshape(im, (1, 3, 256, 144))
            x = np.array(im)
            x = torch.from_numpy(x).type(torch.FloatTensor)
            output = model(x)
            try:
                os.remove("file.txt")
            except:
                pass
            output = output.data.numpy().tolist()
            output = output[0]
            output = output.index(max(output))
            %store feedback[output] >> file.txt
            os.remove("../../../../Downloads/" + file)